In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import os
from pathlib import Path

import ase
import numpy as np

from mcmc.system import SurfaceSystem
from mcmc.utils.misc import get_atoms_batch

np.set_printoptions(precision=3, suppress=True)

Initialize test slabs and parameters. We are testing the dominant surfaces at different $\mu_{Sr}$ values. Starting from the DL-TiO2 termination at low $\mu_{Sr}$, we have the SL-TiO2 termination at mid $\mu_{Sr}$, and the SL-SrO at high $\mu_{Sr}$.

In [3]:
# Load prepared slabs
offset_data_path = os.path.join(
    "../tutorials",
    "data/nff",
    "offset_data.json",
)

ref_slab_files = [
    "data/SrTiO3_001/O44Sr12Ti16.cif",
    "data/SrTiO3_001/O36Sr12Ti12.cif",
    "data/SrTiO3_001/O40Sr16Ti12.cif",
]

ref_slabs = [ase.io.read(f) for f in ref_slab_files]

In [4]:
# Initialize paths
surface_name = "SrTiO3(001)"
run_folder = Path() / surface_name
run_folder.mkdir(parents=True, exist_ok=True)

try:
    with open(offset_data_path, "r") as f:
        offset_data = json.load(f)
except FileNotFoundError as e:
    print("Offset data file not found. Please check you have downloaded the data.")
    raise e

calc_settings = {
    "calc_name": "NFF",
    "optimizer": "BFGS",
    "chem_pots": {"Sr": -2, "Ti": 0, "O": 0},
    "relax_atoms": True,
    "relax_steps": 20,
    "offset": True,
    "offset_data": offset_data,
}

system_settings = {
    "surface_name": surface_name,
    "surface_depth": 1,
    "cutoff": 5.0,
    "near_reduce": 0.01,
    "planar_distance": 1.5,
    "no_obtuse_hollow": True,
    "ads_site_type": "all",
}

Set up NFF Calculator. Here, we are using the same neural network weights from our Zenodo dataset (https://zenodo.org/record/7927039). The ensemble requires an `offset_data.json` file.

In [5]:
import torch
from nff.io.ase_calcs import NeuralFF
from nff.utils.cuda import cuda_devices_sorted_by_free_mem

from mcmc.calculators import EnsembleNFFSurface

DEVICE = f"cuda:{cuda_devices_sorted_by_free_mem()[-1]}" if torch.cuda.is_available() else "cpu"

# requires an ensemble of models in this path and an `offset_data.json` file
nnids = ["model01", "model02", "model03"]
model_dirs = [
    os.path.join(
        "../tutorials",
        "data/nff",
        str(x),
        "best_model",
    )
    for x in nnids
]

models = []
for modeldir in model_dirs:
    m = NeuralFF.from_file(modeldir, device=DEVICE).model
    models.append(m)

nff_surf_calc = EnsembleNFFSurface(models, device=DEVICE)
nff_surf_calc.set(**calc_settings)

/home/dux/NeuralForceField/models
offset data: {'bulk_energies': {'O': -0.17747231201, 'Sr': -0.06043637668, 'SrTiO3': -1.470008697358702}, 'stoidict': {'Sr': 0.49995161381315867, 'Ti': -0.0637500349111578, 'O': -0.31241304903276834, 'offset': -11.324476454433157}, 'stoics': {'Sr': 1, 'Ti': 1, 'O': 3}, 'ref_formula': 'SrTiO3', 'ref_element': 'Ti'} is set from parameters
chemical potentials: {'Sr': -2, 'Ti': 0, 'O': 0} are set from parameters
offset data: {'bulk_energies': {'O': -0.17747231201, 'Sr': -0.06043637668, 'SrTiO3': -1.470008697358702}, 'stoidict': {'Sr': 0.49995161381315867, 'Ti': -0.0637500349111578, 'O': -0.31241304903276834, 'offset': -11.324476454433157}, 'stoics': {'Sr': 1, 'Ti': 1, 'O': 3}, 'ref_formula': 'SrTiO3', 'ref_element': 'Ti'} is set from parameters


Test the reference slabs

In [6]:
ref_slab_batches = [
    get_atoms_batch(
        slab,
        system_settings["cutoff"],
        DEVICE,
        props={"energy": 0, "energy_grad": []},
    )
    for slab in ref_slabs
]

ref_surfs = []
for ref_slab_batch in ref_slab_batches:
    ref_surf = SurfaceSystem(
        ref_slab_batch,
        calc=nff_surf_calc,
        system_settings=system_settings,
        save_folder=run_folder,
    )
    ref_surfs.append(ref_surf)

2024-06-29 18:44:19,366|INFO|Initalizing adsorption sites with settings: {'surface_name': 'SrTiO3(001)', 'surface_depth': 1, 'cutoff': 5.0, 'near_reduce': 0.01, 'planar_distance': 1.5, 'no_obtuse_hollow': True, 'ads_site_type': 'all'}
2024-06-29 18:44:19,499|INFO|Generated adsorption coordinates are: [array([ 7.695,  0.02 , 20.737]), array([ 2.31 ,  6.011, 20.81 ]), array([ 2.311,  2.026, 20.81 ]), array([ 7.695,  4.005, 20.738]), array([ 0.118,  2.017, 21.03 ])]...
2024-06-29 18:44:19,501|INFO|Initializing 64 virtual atoms
2024-06-29 18:44:19,507|INFO|Initial state is [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
2024-06-29 18:44:19,509|INFO|Number of pristine atoms is 72
2024-06-29 18:44:19,515|INFO|Bulk indices are [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 6

      Step     Time          Energy         fmax
BFGS:    0 18:44:20     -570.127991        0.7374
BFGS:    1 18:44:20     -570.142517        0.6409
BFGS:    2 18:44:20     -570.188354        0.1144
BFGS:    3 18:44:20     -570.188721        0.1004
BFGS:    4 18:44:20     -570.189758        0.0112
BFGS:    5 18:44:20     -570.189758        0.0097


2024-06-29 18:44:20,670|INFO|Initalizing adsorption sites with settings: {'surface_name': 'SrTiO3(001)', 'surface_depth': 1, 'cutoff': 5.0, 'near_reduce': 0.01, 'planar_distance': 1.5, 'no_obtuse_hollow': True, 'ads_site_type': 'all'}
2024-06-29 18:44:20,798|INFO|Generated adsorption coordinates are: [array([ 0.   ,  7.941, 18.819]), array([ 1.968,  1.953, 18.776]), array([ 1.968,  0.144, 18.735]), array([ 0.   ,  3.956, 18.819]), array([ 1.968,  5.938, 18.776])]...
2024-06-29 18:44:20,800|INFO|Initializing 64 virtual atoms
2024-06-29 18:44:20,804|INFO|Initial state is [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
2024-06-29 18:44:20,804|INFO|Number of pristine atoms is 60
2024-06-29 18:44:20,809|INFO|Bulk indices are [ 0  1  2  3  4  5  6  8  9 10 11 12 13 14 15 16 17 18 19 20 21 23 24 25
 26 27 28 29 30 31 32 33 34 35 36 38 39 40 41 42 43 44 45 46 47 48 49 50
 51 53 54 55 56 57 58 59]
2024-06-29 18:4

      Step     Time          Energy         fmax
BFGS:    0 18:44:20     -467.525604        0.1416
BFGS:    1 18:44:21     -467.526703        0.1320
BFGS:    2 18:44:21     -467.534149        0.0035


2024-06-29 18:44:21,108|INFO|Initalizing adsorption sites with settings: {'surface_name': 'SrTiO3(001)', 'surface_depth': 1, 'cutoff': 5.0, 'near_reduce': 0.01, 'planar_distance': 1.5, 'no_obtuse_hollow': True, 'ads_site_type': 'all'}
2024-06-29 18:44:21,227|INFO|Generated adsorption coordinates are: [array([ 1.884,  0.412, 20.718]), array([ 5.926,  7.597, 20.717]), array([ 7.732,  6.047, 20.416]), array([ 1.972,  3.644, 20.709]), array([ 6.014,  4.393, 20.711])]...
2024-06-29 18:44:21,229|INFO|Initializing 48 virtual atoms
2024-06-29 18:44:21,233|INFO|Initial state is [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
2024-06-29 18:44:21,233|INFO|Number of pristine atoms is 68
2024-06-29 18:44:21,242|INFO|Bulk indices are [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 62 65 66 67]
2024-06-29 18:44:21,243

      Step     Time          Energy         fmax
BFGS:    0 18:44:21     -518.694092        0.7792
BFGS:    1 18:44:21     -518.721985        0.6676
BFGS:    2 18:44:21     -518.753479        1.1361
BFGS:    3 18:44:21     -518.766785        0.3977
BFGS:    4 18:44:21     -518.778076        0.2571
BFGS:    5 18:44:21     -518.780823        0.2640
BFGS:    6 18:44:21     -518.782776        0.0935
BFGS:    7 18:44:22     -518.783081        0.0494
BFGS:    8 18:44:22     -518.783203        0.0442
BFGS:    9 18:44:22     -518.783386        0.0503
BFGS:   10 18:44:22     -518.783447        0.0373
BFGS:   11 18:44:22     -518.783508        0.0181
BFGS:   12 18:44:22     -518.783508        0.0108
BFGS:   13 18:44:22     -518.783630        0.0085


The below should output:
```
energy of reference slab is [35.931]
energy of reference slab is [12.478]
energy of reference slab is [-4.876]
```

In [7]:
for surf in ref_surfs:
    surf_energy = surf.get_surface_energy()
    print(f"energy of reference slab is {surf_energy}")

energy of reference slab is [35.931]
energy of reference slab is [12.478]
energy of reference slab is [-4.876]
